In [3]:
pip install pyaudio

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [5]:
pip install gtts SpeechRecognition spacy ipython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import spacy
import speech_recognition as sr
from gtts import gTTS
import IPython.display as ipd
import re

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Sample order database
orders = {
    "1123": {"status": "Dispatched", "delivery": "4-7 days", "cancellable": False},
    "1124": {"status": "Processing", "delivery": "Not Dispatched", "cancellable": True},
}

# Common user requests
greetings = ["hi", "hello", "hey", "good morning", "good evening", "nice to meet you"]
complaints = ["scratches", "damaged", "wrong item", "not working", "delayed", "missing"]
policy_keywords = ["policy", "policies", "terms", "return", "refund"]
payment_keywords = ["payment", "method", "pay", "credit card", "paypal", "apple pay"]
help_keywords = ["help", "support", "assist"]
agent_keywords = ["agent", "customer service", "representative", "talk to someone"]
discount_keywords = ["discount", "offer", "coupon", "deals"]
exit_keywords = ["bye", "thank you", "thanks", "no more help", "appreciate it"]

# Speak function using Google TTS
def speak(text):
    print(f"Bot: {text}")
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    return ipd.Audio("response.mp3", autoplay=True)

# Speech recognition function
def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening... Speak now.")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source, timeout=5)
            user_input = recognizer.recognize_google(audio).lower()
            print(f"User: {user_input}")  
            return user_input
        except (sr.UnknownValueError, sr.RequestError, sr.WaitTimeoutError):
            return None  

# Extract order number from user input
def extract_order_number(user_input):
    if user_input is None:
        return None
    match = re.search(r"\b\d{3,6}\b", user_input)  
    return match.group() if match else None

# NLP-based intent detection
def get_intent(user_input):
    if user_input is None:
        return None
    doc = nlp(user_input)
    for token in doc:
        if token.lemma_ in ["cancel", "cancellation"]:
            return "cancel_order"
        elif token.lemma_ in ["status", "track", "check", "order"]:
            return "order_status"
        elif token.lemma_ in ["help", "support", "agent"]:
            return "connect_agent"
        elif token.lemma_ in ["payment", "method", "pay"]:
            return "payment_info"
        elif token.lemma_ in ["return", "refund"]:
            return "return_policy"
        elif token.lemma_ in ["delivery", "shipping", "time"]:
            return "delivery_info"
        elif token.lemma_ in ["discount", "offer", "coupon"]:
            return "discount_info"
        elif token.lemma_ in ["available", "stock", "product"]:
            return "product_availability"
    return "unknown"

# AI Voice Bot Logic
def handle_query():
    speak("Hi there! Welcome to XYZ. My name is Tara. How can I assist you today?")
    
    order_num = None  
    consecutive_failures = 0  

    while True:
        user_input = listen()

        if user_input is None:
            consecutive_failures += 1
            if consecutive_failures >= 2:
                speak("I'm having trouble understanding you. Would you like to type instead?")
                user_input = input("Type your query: ").lower()  
                consecutive_failures = 0  

        if user_input is None or user_input.strip() == "":
            speak("I couldn't hear you properly. Could you say that again?")
            continue

        # Exit when user says "thank you" or similar phrases
        if any(word in user_input for word in exit_keywords):
            speak("You're welcome! Thank you for using XYZ support. Have a great day!")
            return  # Exits immediately

        # Handle agent connection
        if any(word in user_input for word in agent_keywords):
            speak("I will connect you to an agent now. Please hold on.")
            return  # Exits immediately after transferring to an agent

        # Handle greetings
        if any(greet in user_input for greet in greetings):
            speak("Nice to meet you! How can I assist you today?")
            continue

        # Handle complaints
        if any(word in user_input for word in complaints):
            speak("I'm sorry to hear that. Would you like to start a return or talk to an agent?")
            continue

        # Handle policies
        if any(word in user_input for word in policy_keywords):
            speak("Our return policy allows returns within 30 days. Refunds are processed within 5-7 business days.")
            continue

        # Handle payment information
        if any(word in user_input for word in payment_keywords):
            if "apple pay" in user_input:
                speak("Currently, we do not support Apple Pay, but we are working on adding it soon.")
            else:
                speak("We accept credit cards, PayPal, and XYZ Wallet. You can also choose cash on delivery for selected products.")
            continue

        # Handle discounts
        if any(word in user_input for word in discount_keywords):
            speak("We have discounts on electronics and fashion this week! You can check the 'Deals' section of our XYZ app for the latest offers.")
            continue

        # Extract order number
        extracted_order = extract_order_number(user_input)
        if extracted_order:
            order_num = extracted_order

        # Order-related interactions
        if "order" in user_input or order_num:
            if order_num and order_num in orders:
                order_info = orders[order_num]
                speak(f"Your order {order_num} is {order_info['status']} and expected delivery: {order_info['delivery']}. What would you like to do next?")
                
                user_response = listen()
                if user_response is None:
                    speak("I didn't hear that. Could you repeat?")
                    continue

                intent = get_intent(user_response)

                if intent == "cancel_order":
                    if order_info["cancellable"]:
                        speak("Are you sure you want to cancel your order?")
                        confirm = listen()
                        if confirm and "yes" in confirm:
                            speak("Your order has been successfully canceled.")
                        else:
                            speak("No problem! Let me know if you need anything else.")
                    else:
                        speak(f"Your order is already dispatched and will arrive in {order_info['delivery']}. You can return it after delivery.")
                
                elif intent == "order_status":
                    speak(f"Your order {order_num} is currently {order_info['status']} and expected delivery: {order_info['delivery']}.")

                else:
                    speak("For more details, please check your account on the XYZ app.")

            else:
                speak("I couldn't find an order associated with that number. Please check and try again.")

        else:
            speak("I'm sorry, I didn't understand that. Could you rephrase your question?")

# Run the chatbot
handle_query()


Bot: Hi there! Welcome to XYZ. My name is Tara. How can I assist you today?
Listening... Speak now.
Bot: I couldn't hear you properly. Could you say that again?
Listening... Speak now.
User: may i know about my order
Bot: I couldn't find an order associated with that number. Please check and try again.
Listening... Speak now.
User: my order number is 1124
Bot: Your order 1124 is Processing and expected delivery: Not Dispatched. What would you like to do next?
Listening... Speak now.
User: can i cancel my order
Bot: Are you sure you want to cancel your order?
Listening... Speak now.
User: i don't want to cancel my order
Bot: No problem! Let me know if you need anything else.
Listening... Speak now.
User: what type of payments are accepted by the xyz
Bot: We accept credit cards, PayPal, and XYZ Wallet. You can also choose cash on delivery for selected products.
Listening... Speak now.
User: is there any offers for me
Bot: We have discounts on electronics and fashion this week! You can ch